## pitchとplayerの前処理を統合
#### 22A
- all_pitcher_17, all_player_17
- 22相当に戻す

#### 23
- all_pitcher_16, all_player_16
- 2017年の集計がない選手はNaNに

#### 22
- all_pitcher_15, all_player_15
- 2017年の結果を1/4ずつ集計

#### 21
- 19相当に戻す, 球種サブモデルの予測方法変更

#### 20
- player同士の組み合わせを削除

#### 19
- all_pitch_14, all_pitcher_14.f, all_player_14

#### 18
- all_pitch_13, all_pitcher_13.f, all_player_13

#### 17
- all_pitch_12

#### 16
- all_pitch_11, all_pitcher_12.f, all_player_12

#### 15
- all_pitch_10

#### 14
- all_pitch_9 ,all_player_11, all_pitcher_player_1.f

#### 13
- all_pitch_8 ,all_player_10

In [14]:
import pandas as pd
import numpy as np
import feather
pd.set_option('display.max_Columns', 300)

### 管理番号

In [15]:
model_No = '31'

### sub-modelを使用するかどうか

In [16]:
use_sub_model = False

### n/4

In [17]:
sample_No = 1

In [18]:
ALL_PITCH = 'intermediate/pitch/all_pitch.f'
ALL_PITCHER = 'intermediate/player/all_pitcher_{}.f'.format(sample_No)
ALL_CATCHER = 'intermediate/player/all_catcher_{}.f'.format(sample_No)
ALL_PLAYER = 'intermediate/player/all_player_{}.f'.format(sample_No)

SUB_BALL = 'intermediate/{}/ball_predict_{}_{}.f'.format(model_No, model_No, sample_No)
SUB_COURSE = 'intermediate/{}/course_predict_{}_{}.f'.format(model_No, model_No, sample_No)

OUTPUT = 'intermediate/{}/all_merge_{}_{}.f'.format(model_No, model_No, sample_No)
OUTPUT_SUB = 'intermediate/{}/all_merge_{}_{}_sub.f'.format(model_No, model_No, sample_No)

In [19]:
all_pitch = pd.read_feather(ALL_PITCH)
all_pitch.shape

(778767, 119)

In [20]:
all_pitcher = pd.read_feather(ALL_PITCHER)
all_pitcher.shape

(2832, 55)

In [21]:
all_catcher = pd.read_feather(ALL_CATCHER)
all_catcher.shape

(1112, 52)

In [22]:
all_player = pd.read_feather(ALL_PLAYER)
all_player.shape

(2757, 21)

In [23]:
all_catcher.head()

,年度,選手ID,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,pit_bat,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,course_9,course_10,course_11,course_12,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball
0,2017,11343,0,183,86,5.0,4500,0,0,0,1,41,23.0,26.5,1.152174,609.5,195.652174,103500.0,25.680074,R_L,0.459101,0.081565,0.137648,0.082747,0.111777,0.055196,0.018668,0.053298,630.196078,0.061388,0.088208,0.069726,0.043995,0.058027,0.042190,0.033339,0.048569,0.036605,0.152287,0.063395,0.153858,0.148413,0.138721,0.215681,0.194804,0.148522,0.302271,0.219322,0.306145,0.144213,0.118513,0.211808
1,2017,11343,0,183,86,5.0,4500,0,0,0,1,41,23.0,26.5,1.152174,609.5,195.652174,103500.0,25.680074,R_R,0.450897,0.085432,0.214673,0.068968,0.085571,0.028021,0.005557,0.060881,709.196078,0.036343,0.045066,0.030427,0.050444,0.063424,0.043504,0.056982,0.083829,0.071180,0.097031,0.091847,0.070627,0.259297,0.143770,0.188877,0.192319,0.145110,0.329923,0.111836,0.167657,0.157372,0.211991,0.351143
2,2017,11343,0,183,86,5.0,4500,0,0,0,1,41,23.0,26.5,1.152174,609.5,195.652174,103500.0,25.680074,L_L,0.512540,0.052102,0.280525,0.051336,0.025138,0.030908,0.009714,0.037737,178.800000,0.068350,0.110974,0.094902,0.043094,0.059991,0.036442,0.025315,0.034087,0.027611,0.099611,0.089427,0.250854,0.059343,0.136760,0.189038,0.205051,0.158954,0.310197,0.274226,0.350465,0.139527,0.087013,0.148770
3,2017,11343,0,183,86,5.0,4500,0,0,0,1,41,23.0,26.5,1.152174,609.5,195.652174,103500.0,25.680074,L_R,0.460519,0.050544,0.166626,0.069244,0.048656,0.148380,0.025481,0.030550,321.449438,0.040442,0.058024,0.053829,0.039638,0.055926,0.044112,0.047537,0.063616,0.054493,0.073264,0.124856,0.174770,0.169492,0.127617,0.198119,0.177567,0.152435,0.344262,0.152295,0.248034,0.139676,0.165647,0.294348
4,2017,11726,0,177,85,1.0,3700,0,0,0,1,36,19.0,28.5,1.500000,541.5,194.736842,70300.0,27.131412,R_L,0.432432,0.054054,0.270270,0.027027,0.162162,0.000000,0.000000,0.054054,37.000000,0.081081,0.081081,0.054054,0.054054,0.081081,0.081081,0.000000,0.027027,0.027027,0.189189,0.000000,0.135135,0.189189,0.135135,0.189189,0.189189,0.162162,0.324324,0.216216,0.324324,0.216216,0.054054,0.189189


### Join
- pitch.投手ID/年度 - player.選手ID/年度
- pitch.打者ID/年度 - player.選手ID/年度
- pitch.捕手ID/年度 - player.選手ID/年度

In [24]:
merge_all = pd.merge(all_pitch, all_pitcher, left_on=['投手ID', '年度', 'pit_bat'], right_on=['選手ID', '年度', 'pit_bat'], how='left')

In [25]:
merge_all = pd.merge(merge_all, all_player, left_on=['打者ID', '年度'], right_on=['選手ID', '年度'], how='left', suffixes=['_pit', '_bat'])

In [26]:
merge_all = pd.merge(merge_all, all_catcher, left_on=['捕手ID', '年度', 'pit_bat'], right_on=['選手ID', '年度', 'pit_bat'], how='left', suffixes=['', '_cat'])

In [27]:
merge_all.head()

,データ内連番,ball,course,年度,試合ID,試合内連番,試合内投球数,ホームチームID,アウェイチームID,イニング,イニング内打席数,打席内投球数,投手ID,投手チームID,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打順,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,捕手ID,pit_bat,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_course_0,bc_course_1,bc_course_2,bc_course_3,bc_course_4,bc_course_5,bc_course_6,bc_course_7,bc_course_8,bc_course_9,bc_course_10,bc_course_11,bc_course_12,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,opening_date,game_date,date_from_opening,start_time,game_time,elapsed_time,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,選手ID_pit,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,course_9,course_10,course_11,course_12,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball,選手ID_bat,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,選手ID,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,straight_cat,curve_cat,slider_cat,shoot_cat,fork_cat,changeup_cat,sinker_cat,cutball_cat,total_cat,course_0_cat,course_1_cat,course_2_cat,course_3_cat,course_4_cat,course_5_cat,course_6_cat,course_7_cat,course_8_cat,course_9_cat,course_10_cat,course_11_cat,course_12_cat,high_str_cat,high_ball_cat,mid_str_cat,low_str_cat,low_ball_cat,left_str_cat,left_ball_cat,center_str_cat,right_str_cat,right_ball_cat
0,0,0.0,0.0,2017,2017033101,1,1,1,4,1,1,1,1500001,1,1,1,1,1,900410,4,1,1,0,0,0,0,0,___,1300027,R_L,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,2017-03-31,2017-03-31,0,2020-06-13 18:03:00,2020-06-13 18:03:00,00:00:00,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,1500001,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.350694,0.18750,0.180556,0.232639,0.0,0.048611,0.0,0.0,288.0,0.093750,0.093750,0.038194,0.052083,0.048611,0.076389,0.045139,0.065972,0.034722,0.149306,0.069444,0.111111,0.121528,6.0,40.0,83.0,0.190972,0.218750,0.208333,0.149306,0.232639,0.225694,0.260417,0.177083,0.145833,0.190972,900410,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,160.0,35.0,1300027.0,0.0,178.0,83.0,1.0,5000.0,0.0,1.0,1.0,0.0,28.0,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,0.360597,0.088193,0.202703,0.167852,0.091750,0.044097,0.0,0.044808,1406.0,0.064011,0.093172,0.076102,0.039829,0.051209,0.036984,0.037696,0.048364,0.034139,0.134424,0.071835,0.162162,0.150071,0.14153

### player同士の組み合わせ
- 年棒の差
- 現役年数の差
- 年齢の差
- salary_yearの差
- salary_x_yearの差
- BMIの差

In [196]:
merge_all['salary_dif_p-b'] = merge_all['salary_pit'] - merge_all['salary_bat']
merge_all['play_year_dif_p-b'] = merge_all['play_year_pit'] - merge_all['play_year_bat']
merge_all['age_dif_p-b'] = merge_all['age_pit'] - merge_all['age_bat']
merge_all['salary_year_dif_p-b'] = merge_all['salary_year_pit'] - merge_all['salary_year_bat']
merge_all['salary_x_year_dif_p-b'] = merge_all['salary_x_year_pit'] - merge_all['salary_x_year_bat']
merge_all['rank_year_dif_p-b'] = merge_all['rank_year_pit'] - merge_all['rank_year_bat']
merge_all['rank_x_year_dif_p-b'] = merge_all['rank_x_year_pit'] - merge_all['rank_x_year_bat']
merge_all['bmi_dif_p-b'] = merge_all['bmi_pit'] - merge_all['bmi_bat']

merge_all['salary_dif_p-c'] = merge_all['salary_pit'] - merge_all['salary']
merge_all['play_year_dif_p-c'] = merge_all['play_year_pit'] - merge_all['play_year']
merge_all['age_dif_p-c'] = merge_all['age_pit'] - merge_all['age']
merge_all['salary_year_dif_p-c'] = merge_all['salary_year_pit'] - merge_all['salary_year']
merge_all['salary_x_year_dif_p-c'] = merge_all['salary_x_year_pit'] - merge_all['salary_x_year']
merge_all['rank_year_dif_p-c'] = merge_all['rank_year_pit'] - merge_all['rank_year']
merge_all['rank_x_year_dif_p-c'] = merge_all['rank_x_year_pit'] - merge_all['rank_x_year']
merge_all['bmi_dif_p-c'] = merge_all['bmi_pit'] - merge_all['bmi']

merge_all['salary_dif_b-c'] = merge_all['salary_bat'] - merge_all['salary']
merge_all['play_year_dif_b-c'] = merge_all['play_year_bat'] - merge_all['play_year']
merge_all['age_dif_b-c'] = merge_all['age_bat'] - merge_all['age']
merge_all['salary_year_dif_b-c'] = merge_all['salary_year_bat'] - merge_all['salary_year']
merge_all['salary_x_year_dif_b-c'] = merge_all['salary_x_year_bat'] - merge_all['salary_x_year']
merge_all['rank_year_dif_b-c'] = merge_all['rank_year_bat'] - merge_all['rank_year']
merge_all['rank_x_year_dif_b-c'] = merge_all['rank_x_year_bat'] - merge_all['rank_x_year']
merge_all['bmi_dif_b-c'] = merge_all['bmi_bat'] - merge_all['bmi']

### 球種の組合せ

In [28]:
ball_kind = ['straight', 'curve', 'slider', 'shoot', 'fork', 'changeup', 'sinker', 'cutball']

In [29]:
for ball in ball_kind:
    target = 'sub_' + ball
    bc_src = 'bc_' + ball
    merge_all[target] = merge_all[bc_src] - merge_all[ball]

In [30]:
for ball in ball_kind:
    target = 'div_' + ball
    bc_src = 'bc_' + ball
    merge_all[target] = merge_all[bc_src] / merge_all[ball]

In [31]:
for ball in ball_kind:
    target = 'mul_' + ball
    bc_src = 'bc_' + ball
    merge_all[target] = merge_all[bc_src] * merge_all[ball]

In [32]:
for ball in ball_kind:
    target = 'ave_' + ball
    bc_src = 'bc_' + ball
    merge_all[target] = (merge_all[bc_src] + merge_all[ball])/2

In [33]:
for ball in ball_kind:
    target = 'rate_' + ball
    ave_src = 'ave_' + ball
    merge_all[target] = merge_all[ave_src] /(1-merge_all['ave_straight'])

In [40]:
merge_all.head()

,データ内連番,ball,course,年度,試合ID,試合内連番,試合内投球数,ホームチームID,アウェイチームID,イニング,イニング内打席数,打席内投球数,投手ID,投手チームID,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打順,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,捕手ID,pit_bat,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_course_0,bc_course_1,bc_course_2,bc_course_3,bc_course_4,bc_course_5,bc_course_6,bc_course_7,bc_course_8,bc_course_9,bc_course_10,bc_course_11,bc_course_12,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,opening_date,game_date,date_from_opening,start_time,game_time,elapsed_time,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,選手ID_pit,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,course_0,course_1,course_2,course_3,...,course_10,course_11,course_12,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball,選手ID_bat,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,選手ID,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,straight_cat,curve_cat,slider_cat,shoot_cat,fork_cat,changeup_cat,sinker_cat,cutball_cat,total_cat,course_0_cat,course_1_cat,course_2_cat,course_3_cat,course_4_cat,course_5_cat,course_6_cat,course_7_cat,course_8_cat,course_9_cat,course_10_cat,course_11_cat,course_12_cat,high_str_cat,high_ball_cat,mid_str_cat,low_str_cat,low_ball_cat,left_str_cat,left_ball_cat,center_str_cat,right_str_cat,right_ball_cat,sub_straight,sub_curve,sub_slider,sub_shoot,sub_fork,sub_changeup,sub_sinker,sub_cutball,div_straight,div_curve,div_slider,div_shoot,div_fork,div_changeup,div_sinker,div_cutball,mul_straight,mul_curve,mul_slider,mul_shoot,mul_fork,mul_changeup,mul_sinker,mul_cutball,ave_straight,ave_curve,ave_slider,ave_shoot,ave_fork,ave_changeup,ave_sinker,ave_cutball,rate_straight,rate_curve,rate_slider,rate_shoot,rate_fork,rate_changeup,rate_sinker,rate_cutball,sub_p_cstraight,sub_p_ccurve,sub_p_cslider,sub_p_cshoot,sub_p_cfork,sub_p_cchangeup,sub_p_csinker,sub_p_ccutball,sub_p_c_straight,sub_p_c_curve,sub_p_c_slider,sub_p_c_shoot,sub_p_c_fork,sub_p_c_changeup,sub_p_c_sinker,sub_p_c_cutball,sub_bc_c_straight,sub_bc_c_curve,sub_bc_c_slider,sub_bc_c_shoot,sub_bc_c_fork,sub_bc_c_changeup,sub_bc_c_sinker,sub_bc_c_cutball
0,0,0.0,0.0,2017,2017033101,1,1,1,4,1,1,1,1500001,1,1,1,1,1,900410,4,1,1,0,0,0,0,0,___,1300027,R_L,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,2017-03-31,2017-03-31,0,2020-06-13 18:03:00,2020-06-13 18:03:00,00:00:00,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0

### 投手と捕手の組合せ

In [37]:
for ball in ball_kind:
    target = 'sub_p_c_' + ball
    cat = ball + '_cat'
    merge_all[target] = merge_all[ball] - merge_all[cat]

In [39]:
for ball in ball_kind:
    target = 'sub_bc_c_' + ball
    cat = ball + '_cat'
    bc_src = 'bc_' + ball
    merge_all[target] = merge_all[bc_src] - merge_all[cat]

In [ ]:
ball_kind_bc = list(map(lambda x: 'bc_' + x, ball_kind))
merge_all.drop(columns=ball_kind, inplace=True)
merge_all.drop(columns=ball_kind_bc, inplace=True)

### コースの組合せ

In [204]:
course_kind = ['course_0', 'course_1', 'course_2', 'course_3', 'course_4', 'course_5', 'course_6', 
                'course_7', 'course_8', 'course_9', 'course_10', 'course_11', 'course_12',
               'high_str', 'high_ball', 'mid_str', 'low_str', 'low_ball', 
               'left_str', 'left_ball', 'center_str', 'right_str', 'right_ball']

In [205]:
for course in course_kind:
    target = 'sub_' + course
    bc_src = 'bc_' + course
    merge_all[target] = merge_all[bc_src] - merge_all[course]

In [206]:
for course in course_kind:
    target = 'div_' + course
    bc_src = 'bc_' + course
    merge_all[target] = merge_all[bc_src] / merge_all[course]

In [207]:
for course in course_kind:
    target = 'mul_' + course
    bc_src = 'bc_' + course
    merge_all[target] = merge_all[bc_src] * merge_all[course]

In [208]:
for course in course_kind:
    target = 'ave_' + course
    bc_src = 'bc_' + course
    merge_all[target] = (merge_all[bc_src] + merge_all[course])/2

In [209]:
course_kind_bc = list(map(lambda x: 'bc_' + x, course_kind))
merge_all.drop(columns=course_kind, inplace=True)
merge_all.drop(columns=course_kind_bc, inplace=True)

In [210]:
merge_all.head()

,データ内連番,ball,course,年度,試合ID,試合内連番,試合内投球数,ホームチームID,アウェイチームID,イニング,イニング内打席数,打席内投球数,投手ID,投手チームID,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打順,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,捕手ID,pit_bat,pitch_LR,bat_LR,first,second,third,base_cnt,top_bot,role,pos_pit,opening_date,game_date,date_from_opening,start_time,game_time,elapsed_time,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,選手ID_pit,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,選手ID_bat,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,選手ID,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,...,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,sub_straight,sub_curve,sub_slider,sub_shoot,sub_fork,sub_changeup,sub_sinker,sub_cutball,div_straight,div_curve,div_slider,div_shoot,div_fork,div_changeup,div_sinker,div_cutball,mul_straight,mul_curve,mul_slider,mul_shoot,mul_fork,mul_changeup,mul_sinker,mul_cutball,ave_straight,ave_curve,ave_slider,ave_shoot,ave_fork,ave_changeup,ave_sinker,ave_cutball,rate_straight,rate_curve,rate_slider,rate_shoot,rate_fork,rate_changeup,rate_sinker,rate_cutball,sub_course_0,sub_course_1,sub_course_2,sub_course_3,sub_course_4,sub_course_5,sub_course_6,sub_course_7,sub_course_8,sub_course_9,sub_course_10,sub_course_11,sub_course_12,sub_high_str,sub_high_ball,sub_mid_str,sub_low_str,sub_low_ball,sub_left_str,sub_left_ball,sub_center_str,sub_right_str,sub_right_ball,div_course_0,div_course_1,div_course_2,div_course_3,div_course_4,div_course_5,div_course_6,div_course_7,div_course_8,div_course_9,div_course_10,div_course_11,div_course_12,div_high_str,div_high_ball,div_mid_str,div_low_str,div_low_ball,div_left_str,div_left_ball,div_center_str,div_right_str,div_right_ball,mul_course_0,mul_course_1,mul_course_2,mul_course_3,mul_course_4,mul_course_5,mul_course_6,mul_course_7,mul_course_8,mul_course_9,mul_course_10,mul_course_11,mul_course_12,mul_high_str,mul_high_ball,mul_mid_str,mul_low_str,mul_low_ball,mul_left_str,mul_left_ball,mul_center_str,mul_right_str,mul_right_ball,ave_course_0,ave_course_1,ave_course_2,ave_course_3,ave_course_4,ave_course_5,ave_course_6,ave_course_7,ave_course_8,ave_course_9,ave_course_10,ave_course_11,ave_course_12,ave_high_str,ave_high_ball,ave_mid_str,ave_low_str,ave_low_ball,ave_left_str,ave_left_ball,ave_center_str,ave_right_str,ave_right_ball
0,0,0.0,0.0,2017,2017033101,1,1,1,4,1,1,1,1500001,1,1,1,1,1,900410,4,1,1,0,0,0,0,0,___,1300027,R_L,0.0,1.0,0,0,0,0,0,1,0,2017-03-31,2017-03-31,0,2020-05-31 18:03:00,2020-05-31 18:03:00,00:00:00,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,1500001,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,40

### ダミー変数

In [207]:
merge_all = pd.get_dummies(merge_all, columns=['pit_bat'])

### 不要な列を削除

In [208]:
merge_all.drop(columns=[
    '選手ID_pit', '選手ID_bat', '選手ID',
    '試合内連番',
    '年度', 
    '試合ID', 
    'ホームチームID', 'アウェイチームID', 
    '投手ID', '投手チームID', 
    '打者ID', '打者チームID', 
    'プレイ前走者状況', 
    '捕手ID', 
    'opening_date', 'game_date',
    'start_time', 'game_time', 'elapsed_time'
], inplace=True)

### Rename

In [209]:
merge_all.rename(columns={
    'データ内連番': 'No',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [210]:
print(merge_all.shape)
merge_all.head()

(778767, 190)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,elapsed_batter,ave_elapsed_game,ave_elapsed_inning,ave_elapsed_batter,diff_elapsed_batter,diff_elapsed_inning,diff_elapsed_game,sayonara,extention,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_count_sum,pre_ball_foul,pre_ball_ball,pre_ball_strike,pre_foul_sum,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,firm_pit,height_pit,weight_pit,draft_order_pit,salary_pit,foreigner_pit,company_pit,univ_pit,highsch_pit,age_pit,play_year_pit,salary_rank_pit,rank_year_pit,rank_x_year_pit,salary_year_pit,salary_x_year_pit,bmi_pit,curve,slider,shoot,fork,changeup,sinker,cutball,total,pit_game_cnt,pit_inning_cnt,pit_batter_cnt,high_str,high_ball,mid_str,low_str,low_ball,left_str,left_ball,center_str,right_str,right_ball,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_rank_bat,rank_year_bat,rank_x_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.237468,0.325101,0.209547,0.145227,0.092978,0.027829,0.113620,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,0,0.000000,NaN,0,0.000000,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.368821,0.292776,0.114068,0.083650,0.04943,0.0,0.0,502.0,8.0,57.0,120.0,0.159363,0.201195,0.219124,0.191235,0.229084,0.183267,0.239044,0.205179,0.181275,0.191235,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,66.0,16.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.210923,0.247198,0.177502,0.320359,0.147952,0.039943,0.116772,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,0.242405,0,0,0,0,0,1,0,0,0.200000,0.200000,0,0.200000,2,0.200000,0.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0.0,0.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,196,100,NaN,24000,1,0,0,0,29,6.0,5.5,0.916667,33.0,4000.0,144000.0,26.03082,0.368821,0.292776,0.114068,0.083650,0.04943,0.0,0.0,502.0,8.0,57.0,120.0,0.159363,0.201195,0.219124,0.191235,0.229084,0.183267,0.239044,0.205179,0.181275,0.191235,0,176,74,5.0,15000,0,1,1,0,32,8.0,2.5,0.312500,20.0,1875.0,120000.0,23.889463,66.0,16.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-

### 出力(sub-modelなし)

In [211]:
if not use_sub_model:
    merge_all.to_feather(OUTPUT)
    print(OUTPUT)

### 予測結果を特徴量に加える

In [212]:
if use_sub_model:
    sub_ball = pd.read_feather(SUB_BALL)
    sub_course = pd.read_feather(SUB_COURSE)
    print(sub_ball.shape)
    print(sub_course.shape)

(701651, 8)
(701651, 13)


In [213]:
if use_sub_model:
    merge_sub = pd.concat([merge_all, sub_ball], axis=1)
    merge_sub = pd.concat([merge_sub, sub_course], axis=1)
    print(merge_sub.shape)

(778767, 211)


### 出力(sub-modelあり)

In [214]:
if use_sub_model:
    merge_sub.to_feather(OUTPUT_SUB)
    print(OUTPUT_SUB)

intermediate/22A/all_merge_22A_4_sub.f


### GCS
#### local->GCS

In [215]:
GCS = 'gs://yasunobu_baseball/'

In [216]:
'gsutil cp ./{} {}intermediate/'.format(OUTPUT, GCS)

'gsutil cp ./intermediate/22A/all_merge_22A_4.f gs://yasunobu_baseball/intermediate/'

In [217]:
'gsutil cp ./{} {}intermediate/'.format(OUTPUT_SUB, GCS)

'gsutil cp ./intermediate/22A/all_merge_22A_4_sub.f gs://yasunobu_baseball/intermediate/'

#### GCS->GCE

In [218]:
'gsutil cp {}{} ~/work/baseball/intermediate'.format(GCS, OUTPUT)

'gsutil cp gs://yasunobu_baseball/intermediate/22A/all_merge_22A_4.f ~/work/baseball/intermediate'